In [ ]:
pip install fsspec==2024.10.0

In [ ]:
pip check

ipython 7.34.0 requires jedi, which is not installed.
pygobject 3.42.1 requires pycairo, which is not installed.


In [ ]:
pip install jedi pycairo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.5/661.5 kB 13.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
pip install fsspec[http]==2024.9.0

In [ ]:
!pip install -q torch
!pip install -q git+https://github.com/huggingface/transformers #huggingface transformers for downloading models weights
!pip install -q datasets #huggingface datasets to download and manipulate datasets
!pip install -q peft #Parameter efficient finetuning - for qLora Finetuning
!pip install -q bitsandbytes #For Model weights quantisation
!pip install -q trl #Transformer Reinforcement Learning - For Finetuning using Supervised Fine-tuning
!pip install -q wandb -U #Used to monitor the model score during training

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 7.0 MB/s eta 0:00:00


In [ ]:
import json
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer

In [ ]:
notebook_login()

In [ ]:
pip install transformers

In [ ]:
!pip install transformers huggingface_hub

In [ ]:
!ls /content

sample_data  transformed_data.json


In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/content/transformed_data.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(dataset[:-1])  # Print the first 5 rows to verify the structure

{'instruction': ['Provide details for NE_Name: NE_Z and NE_Type: Router.', 'Provide details for NE_Name: NE_E and NE_Type: Gateway.', 'Provide details for NE_Name: NE_V and NE_Type: Hub.', 'Provide details for NE_Name: NE_W and NE_Type: Hub.', 'Provide details for NE_Name: NE_V and NE_Type: Firewall.', 'Provide details for NE_Name: NE_E and NE_Type: Gateway.', 'Provide details for NE_Name: NE_U and NE_Type: Switch.', 'Provide details for NE_Name: NE_H and NE_Type: Gateway.', 'Provide details for NE_Name: NE_Y and NE_Type: Hub.', 'Provide details for NE_Name: NE_I and NE_Type: Switch.', 'Provide details for NE_Name: NE_X and NE_Type: Router.', 'Provide details for NE_Name: NE_W and NE_Type: Gateway.', 'Provide details for NE_Name: NE_T and NE_Type: Firewall.', 'Provide details for NE_Name: NE_H and NE_Type: Router.', 'Provide details for NE_Name: NE_Y and NE_Type: Router.', 'Provide details for NE_Name: NE_W and NE_Type: Router.', 'Provide details for NE_Name: NE_L and NE_Type: Firewall

In [ ]:
# Convert the entire dataset to a pandas DataFrame
df = dataset.to_pandas()

# Preview the DataFrame
print(df.head())


                                         instruction  \
0  Provide details for NE_Name: NE_Z and NE_Type:...   
1  Provide details for NE_Name: NE_E and NE_Type:...   
2  Provide details for NE_Name: NE_V and NE_Type:...   
3  Provide details for NE_Name: NE_W and NE_Type:...   
4  Provide details for NE_Name: NE_V and NE_Type:...   

                                               input  \
0  {"IP_Address": "192.168.1.152", "Location": "B...   
1  {"IP_Address": "192.168.1.185", "Location": "L...   
2  {"IP_Address": "192.168.1.34", "Location": "Li...   
3  {"IP_Address": "192.168.1.83", "Location": "Ne...   
4  {"IP_Address": "192.168.1.55", "Location": "Sp...   

                                              output  
0  NE 'NE_Z' (Router) is located in Birmingham wi...  
1  NE 'NE_E' (Gateway) is located in Laredo with ...  
2  NE 'NE_V' (Hub) is located in Lincoln with an ...  
3  NE 'NE_W' (Hub) is located in New Orleans with...  
4  NE 'NE_V' (Firewall) is located in Spokane wit..

In [ ]:
def convert_dataset(row):
    instruction = row["instruction"]
    output = row["output"]
    prompt = f"<s>[INST] {instruction} [/INST] {output} </s>"
    return {'text': prompt}

# Iterate over the rows of the DataFrame, passing each row as a dictionary
converted_data = [convert_dataset(row) for _, row in df.iterrows()]

# Convert the list of dictionaries back to a Dataset object
train_dataset = Dataset.from_pandas(pd.DataFrame(converted_data))

# Preview the first few items in the dataset
print(train_dataset[:5])


{'text': ["<s>[INST] Provide details for NE_Name: NE_Z and NE_Type: Router. [/INST] NE 'NE_Z' (Router) is located in Birmingham with an IP address of 192.168.1.152 and was configured on 26-12-2024. </s>", "<s>[INST] Provide details for NE_Name: NE_E and NE_Type: Gateway. [/INST] NE 'NE_E' (Gateway) is located in Laredo with an IP address of 192.168.1.185 and was configured on 23-12-2024. </s>", "<s>[INST] Provide details for NE_Name: NE_V and NE_Type: Hub. [/INST] NE 'NE_V' (Hub) is located in Lincoln with an IP address of 192.168.1.34 and was configured on 08-03-2024. </s>", "<s>[INST] Provide details for NE_Name: NE_W and NE_Type: Hub. [/INST] NE 'NE_W' (Hub) is located in New Orleans with an IP address of 192.168.1.83 and was configured on 11-04-2024. </s>", "<s>[INST] Provide details for NE_Name: NE_V and NE_Type: Firewall. [/INST] NE 'NE_V' (Firewall) is located in Spokane with an IP address of 192.168.1.55 and was configured on 01-05-2024. </s>"]}


In [ ]:
new_model = "mistralai-Code-Instruct" #set the name of the new model

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
import torch
print(torch.cuda.is_available())


True


In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load the base model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)

base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

# Load MitsralAi tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
eval_prompt = """Print hello world in python c and c++"""

# import random
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

base_model.eval()
with torch.no_grad():
    print(tokenizer.decode(base_model.generate(**model_input, max_new_tokens=256, pad_token_id=2)[0], skip_special_tokens=True))

Print hello world in python c and c++

Python:
```python
print("Hello, World!")
```

C:
```c
#include <stdio.h>

int main() {
    printf("Hello, World!\n");
    return 0;
}
```

C++:
```c++
#include <iostream>

int main() {
    std::cout << "Hello, World!\n";
    return 0;
}
```


In [ ]:
# Set LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=100,  # the total number of training steps to perform
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Initialize the SFTTrainer for fine-tuning without max_seq_length or packing
trainer = SFTTrainer(
    model=base_model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    tokenizer=tokenizer,  # Keep tokenizer and remove processing_class
    args=training_arguments
)


<ipython-input-28-27c307a49c64>:42: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/53 [00:00<?, ? examples/s]

In [ ]:
# Start the training process
trainer.train()

# Save the fine-tuned model
trainer.model.save_pretrained(new_model)

Step,Training Loss
25,0.965700
50,0.291700
75,0.268000
100,0.249000


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers fou

In [ ]:
trainer.model.save_pretrained("/content/my_model")

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Specify the directory where you saved the fine-tuned model
model_directory = "/content/my_model"  # e.g., new_model

# Load the fine-tuned model
model = AutoModelForCausalLM.from_pretrained(model_directory)

# Load the tokenizer associated with the model
tokenizer = AutoTokenizer.from_pretrained(model_directory)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]